# Kubernetes Volume I/O Error Troubleshooting Workflow

## Introduction

This notebook documents the workflow of `troubleshoot.py`, a comprehensive system designed to troubleshoot Kubernetes pod volume I/O failures. The system uses a phase-based approach with LangGraph frameworks to systematically identify and resolve storage issues in Kubernetes clusters.

The troubleshooting system consists of four distinct phases:

1. **Phase0**: Information Collection - Builds a Knowledge Graph with system information
2. **Plan Phase**: Generates an Investigation Plan using rule-based and LLM approaches
3. **Phase1**: ReAct Investigation - Executes the Investigation Plan to produce a Fix Plan
4. **Phase2**: Remediation - Executes the Fix Plan to resolve identified issues

This notebook aims to provide a clear understanding of each phase's purpose, components, and implementation, as well as visualize the workflow using Mermaid diagrams.

## Main Workflow

The end-to-end workflow of `troubleshoot.py` orchestrates the four phases sequentially, with each phase building on the outputs of the previous phase:

1. **Phase0** collects comprehensive system information and builds a Knowledge Graph
2. **Plan Phase** analyzes the Knowledge Graph to generate an Investigation Plan
3. **Phase1** executes the Investigation Plan using LangGraph to identify root causes and produce a Fix Plan
4. **Phase2** executes the Fix Plan to remediate the identified issues (can be skipped based on Phase1 output)

The workflow includes decision points where Phase2 may be skipped if no issues are detected or if manual intervention is required.

In [3]:
# Main Workflow Visualization
from IPython.display import display, Markdown

main_workflow = """
```mermaid
graph TD
    Start([Start]) --> Phase0["Phase0: Information Collection\n(Build Knowledge Graph)"]
    Phase0 --> |"Knowledge Graph"| PlanPhase["Plan Phase:\nGenerate Investigation Plan"]
    PlanPhase --> |"Investigation Plan"| Phase1["Phase1: ReAct Investigation\n(Execute Plan, Identify Root Cause)"]
    Phase1 --> Decision{"Skip Phase2?\n(No issues or\nManual intervention)"}
    Decision --> |"Yes"| End([End])
    Decision --> |"No\n(Fix Plan)"| Phase2["Phase2: Remediation\n(Execute Fix Plan)"]
    Phase2 --> End
```
"""

display(Markdown(main_workflow))


```mermaid
graph TD
    Start([Start]) --> Phase0["Phase0: Information Collection
(Build Knowledge Graph)"]
    Phase0 --> |"Knowledge Graph"| PlanPhase["Plan Phase:
Generate Investigation Plan"]
    PlanPhase --> |"Investigation Plan"| Phase1["Phase1: ReAct Investigation
(Execute Plan, Identify Root Cause)"]
    Phase1 --> Decision{"Skip Phase2?
(No issues or
Manual intervention)"}
    Decision --> |"Yes"| End([End])
    Decision --> |"No
(Fix Plan)"| Phase2["Phase2: Remediation
(Execute Fix Plan)"]
    Phase2 --> End
```


## Phase0: Information Collection

### Purpose

Phase0 is responsible for collecting comprehensive diagnostic information about the Kubernetes cluster, focusing on the pod with volume I/O errors. This phase builds a Knowledge Graph that serves as the foundation for the subsequent phases.

### Key Components

- **ComprehensiveInformationCollector**: Collects data from various sources including Kubernetes API, system logs, and hardware diagnostics
- **Knowledge Graph**: A graph-based representation of system entities and their relationships
- **Tool Executors**: Various tools in `/information_collector/tool_executors.py` that collect specific types of information

### Inputs and Outputs

- **Inputs**: Pod name, namespace, volume path
- **Outputs**: 
  - Knowledge Graph with system entities and relationships
  - Collected diagnostic information (pod info, PVC info, PV info, node info, etc.)
  - Issues detected during information collection

## Plan Phase: Investigation Plan Generation

### Purpose

The Plan Phase generates an Investigation Plan that guides the troubleshooting process in Phase1. It analyzes the Knowledge Graph from Phase0 and creates a structured plan with specific steps to investigate the volume I/O issues.

### Key Components

- **InvestigationPlanner**: Orchestrates the plan generation process
- **Rule-based Plan Generator**: Creates initial investigation steps based on predefined rules
- **Static Plan Steps**: Incorporates mandatory steps from `static_plan_step.json`
- **LLM Plan Generator**: Refines and enhances the plan using an LLM without tool invocation

### Three-Step Process

1. **Rule-based preliminary steps**: Generate critical initial investigation steps
2. **Static plan steps integration**: Add mandatory steps from `static_plan_step.json`
3. **LLM refinement**: Refine and supplement the plan using an LLM without tool invocation

### Inputs and Outputs

- **Inputs**: Knowledge Graph from Phase0, pod name, namespace, volume path
- **Outputs**: 
  - Investigation Plan as a formatted string
  - Structured representation of the plan with steps and fallback steps

In [ ]:
# Sample Investigation Plan format

sample_plan = """
Investigation Plan:
Target: Pod default/example-pod, Volume Path: /var/lib/kubelet/pods/123/volumes/kubernetes.io~csi/pvc-abc/mount
Generated Steps: 8 steps

Step 1: Get pod details | Tool: kg_get_entity_info(entity_type='Pod', id='gnode:Pod:default/example-pod') | Expected: Pod configuration and status
Step 2: Check related PVC | Tool: kg_find_path(source_entity_type='Pod', source_id='gnode:Pod:default/example-pod', target_entity_type='PVC', target_id='*') | Expected: Path from Pod to PVC
Step 3: Get PVC details | Tool: kg_get_entity_info(entity_type='PVC', id='gnode:PVC:default/example-pvc') | Expected: PVC configuration and status
Step 4: Check related PV | Tool: kg_find_path(source_entity_type='PVC', source_id='gnode:PVC:default/example-pvc', target_entity_type='PV', target_id='*') | Expected: Path from PVC to PV
Step 5: Get PV details | Tool: kg_get_entity_info(entity_type='PV', id='gnode:PV:pv-example') | Expected: PV configuration and status
Step 6: Check node status | Tool: kg_get_entity_info(entity_type='Node', id='gnode:Node:worker-1') | Expected: Node status and conditions
Step 7: Check for issues | Tool: kg_get_all_issues(severity='primary') | Expected: Primary issues in the system
Step 8: Analyze issues | Tool: kg_analyze_issues() | Expected: Root cause analysis and patterns

Fallback Steps (if main steps fail):
Step F1: Print Knowledge Graph | Tool: kg_print_graph(include_details=True, include_issues=True) | Expected: Complete system visualization | Trigger: kg_get_entity_info_failed
Step F2: Check system logs | Tool: kubectl_logs(pod_name='example-pod', namespace='default') | Expected: Pod logs for error messages | Trigger: kg_get_all_issues_failed
"""

print(sample_plan)

## Phase1: ReAct Investigation

### Purpose

Phase1 executes the Investigation Plan generated in the Plan Phase using a LangGraph ReAct framework. It actively investigates the volume I/O issues by executing tools in a sequential manner, analyzing the results, and producing a Fix Plan.

### Key Components

- **LangGraph StateGraph**: Manages the flow of the investigation process
- **SerialToolNode**: Executes tools sequentially based on the Investigation Plan
- **Knowledge Graph Tools**: Tools for querying and analyzing the Knowledge Graph
- **Kubernetes Tools**: Tools for interacting with the Kubernetes API

### Inputs and Outputs

- **Inputs**: 
  - Investigation Plan from Plan Phase
  - Knowledge Graph and collected information from Phase0
  - Pod name, namespace, volume path
- **Outputs**: 
  - Fix Plan with identified root causes and remediation steps
  - Skip Phase2 flag (true if no issues detected or manual intervention required)

In [2]:
# Phase1 LangGraph Visualization
from IPython.display import display, Markdown

phase1_graph = """
```mermaid
graph TD
    START([Start]) --> call_model["call_model\n(LLM reasoning)"];
    call_model --> tools_condition{"tools_condition\n(Tool requested?)"}
    tools_condition -->|"Tool requested"| serial_tools["SerialToolNode\n(Sequential tool execution)"]
    tools_condition -->|"No tool\nrequested"| check_end["check_end\n(End condition check)"]
    tools_condition -->|"end"| check_end
    serial_tools --> call_model
    check_end -->|"continue"| call_model
    check_end -->|"end"| END([End])
```
"""

display(Markdown(phase1_graph))


```mermaid
graph TD
    START([Start]) --> call_model["call_model
(LLM reasoning)"];
    call_model --> tools_condition{"tools_condition
(Tool requested?)"}
    tools_condition -->|"Tool requested"| serial_tools["SerialToolNode
(Sequential tool execution)"]
    tools_condition -->|"No tool
requested"| check_end["check_end
(End condition check)"]
    tools_condition -->|"end"| check_end
    serial_tools --> call_model
    check_end -->|"continue"| call_model
    check_end -->|"end"| END([End])
```


## Phase2: Remediation

### Purpose

Phase2 executes the Fix Plan generated in Phase1 to remediate the identified issues. It uses a LangGraph workflow similar to Phase1 but with access to additional tools that can modify the system state.

### Key Components

- **LangGraph StateGraph**: Manages the flow of the remediation process
- **SerialToolNode**: Executes remediation tools sequentially based on the Fix Plan
- **Action Tools**: Tools for modifying system state (e.g., fixing file systems, restarting services)
- **Validation Tools**: Tools for validating that the remediation was successful

### Inputs and Outputs

- **Inputs**: 
  - Fix Plan from Phase1
  - Knowledge Graph and collected information from Phase0
- **Outputs**: 
  - Remediation result with actions taken and validation status
  - Recommendations for any remaining issues that require manual intervention

In [3]:
# Phase2 LangGraph Visualization
from IPython.display import display, Markdown

phase2_graph = """
```mermaid
graph TD
    START([Start]) --> call_model["call_model\n(LLM reasoning)"];
    call_model --> tools_condition{"tools_condition\n(Tool requested?)"}
    tools_condition -->|"Tool requested"| serial_tools["SerialToolNode\n(Sequential tool execution)"]
    tools_condition -->|"No tool\nrequested"| check_end["check_end\n(End condition check)"]
    tools_condition -->|"end"| check_end
    serial_tools --> call_model
    check_end -->|"continue"| call_model
    check_end -->|"end"| END([End])
```
"""

display(Markdown(phase2_graph))


```mermaid
graph TD
    START([Start]) --> call_model["call_model
(LLM reasoning)"];
    call_model --> tools_condition{"tools_condition
(Tool requested?)"}
    tools_condition -->|"Tool requested"| serial_tools["SerialToolNode
(Sequential tool execution)"]
    tools_condition -->|"No tool
requested"| check_end["check_end
(End condition check)"]
    tools_condition -->|"end"| check_end
    serial_tools --> call_model
    check_end -->|"continue"| call_model
    check_end -->|"end"| END([End])
```


## Summary

The `troubleshoot.py` system provides a comprehensive approach to troubleshooting Kubernetes pod volume I/O failures through its four-phase workflow:

1. **Phase0** builds a Knowledge Graph with comprehensive system information
2. **Plan Phase** generates a structured Investigation Plan
3. **Phase1** executes the Investigation Plan to identify root causes and produce a Fix Plan
4. **Phase2** executes the Fix Plan to remediate the identified issues

The system leverages LangGraph frameworks for both Phase1 and Phase2, with a focus on sequential tool execution through the SerialToolNode component. This ensures that tools are executed in a specific order, allowing for dependencies between tool calls.

The modular design and phase-based approach make the system extensible and maintainable, following the principles of good software design as outlined in Martin Fowler's *Refactoring* principles.